# problema di trasporto 2

Due depositi (A e B) di carburante riforniscono tre distributori (1,2 e 3).

Ogni deposito ha una disponibilità limitata di carburante e ogni distributore ha una richiesta nota.

I costi di trasporto tra depositi e distributori dipende dalla distanza e dalla quantità di carburante

![img](./img.png)

![tab](./tab.png)

Quanto carburante inviare da ogni deposito a ogni distributore in modo da soddisfare tutte le richieste al costo minimo?


## formulazione generale

Una rete logistica (single-commodity) è formata da n sorgenti $S_1$,...,$S_n$ e $m$ destinazioni $T_1$ ,...,$T_m$.

Dati la disponibilità di prodotto di ogni sorgente, la richiesta di prodotto di ogni destinazione e il costo unitario di trasporto per ogni coppia sorgente-destinazione, quanto prodotto trasportare da ogni sorgente a ogni destinazione in modo da rispettare le disponibilità delle sorgenti e soddisfare le richieste delle destinazioni al costo minimo?

## modello parametrico

![param](./param.png)

![param2](./param2.png)

In [5]:
from pyomo.environ import *
from pyomo.opt import SolverStatus, TerminationCondition

model = ConcreteModel()
model.name = "problema di miscelazione"

i = [ 'deposito A', 'deposito B' ]
d = [ 1000, 1400 ]

j = [ 'milano', 'roma', 'napoli' ]
r = [ 800, 700, 900 ]

c = [[13,   11,  16], 
     [12,   15,  14]]

model.i = Set(initialize=i)
model.j = Set(initialize=j)

c_dict = {}
for i, mi in enumerate(model.i):
    for j, mj in enumerate(model.j):
        c_dict[mi, mj] = c[i][j]

d = {mi: d[i] for i, mi in enumerate(model.i)}
r = {mj: r[j] for j, mj in enumerate(model.j)}

model.d = Param(model.i, initialize=d)
model.r = Param(model.j, initialize=r)

model.c = Param(model.i, model.j, initialize=c_dict)

model.x = Var(model.i, model.j, domain=NonNegativeReals, initialize=0)

obj_expr = sum(sum(model.c[i, j]*model.x[i, j] for j in model.j) for i in model.i)

model.cost = Objective(expr = obj_expr, sense=minimize)

model.constraints = ConstraintList()

for i in model.i:
    model.constraints.add(expr = sum(model.x[i, j] for j in model.j) <= model.d[i])

for j in model.j:
    model.constraints.add(expr = sum(model.x[i, j] for i in model.i) >= model.r[j])

results = SolverFactory('glpk').solve(model)

model.display()

Model problema di miscelazione

  Variables:
    x : Size=6, Index=x_index
        Key                      : Lower : Value : Upper : Fixed : Stale : Domain
        ('deposito A', 'milano') :     0 : 300.0 :  None : False : False : NonNegativeReals
        ('deposito A', 'napoli') :     0 :   0.0 :  None : False : False : NonNegativeReals
          ('deposito A', 'roma') :     0 : 700.0 :  None : False : False : NonNegativeReals
        ('deposito B', 'milano') :     0 : 500.0 :  None : False : False : NonNegativeReals
        ('deposito B', 'napoli') :     0 : 900.0 :  None : False : False : NonNegativeReals
          ('deposito B', 'roma') :     0 :   0.0 :  None : False : False : NonNegativeReals

  Objectives:
    cost : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 30200.0

  Constraints:
    constraints : Size=5
        Key : Lower : Body   : Upper
          1 :  None : 1000.0 : 1000.0
          2 :  None : 1400.0 : 1400.0
          3 : 800